<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/Gemma_Python_Chat_(GPC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'data-assistants-with-gemma:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F64148%2F7669720%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240413%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240413T011414Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D17eafce77f9a84fe05ddb5b39a09a690cadaef2d170f3e757d2faa9211be2ce1d2016b9c91ee3960dd799611545befbd6b351aa8d2b3e595400d7c77c4250010e61db5715092766ace65b740a74156f5254cbac390dfe16b8efd6f6ba3e15a61d067af97074ea2fed5295ba4396486447b681a96974f60bf8c4033b8d2fa70f21266795efb884fd9cb710069f8ba5c282bbaa5cfafc2cd6e74bb14e6059e4a6ff415131fb63342f72d233591ba5de290364271e255ec157f5f647081fdeee189c137e79909bf4107d28368e11110639f1d2b6b2e850ca3ad341816a6830dbb08065c68566e9ee7baa6bf202342313b9a39adca3b6e0850e976414d18ce58503e,pythonquestions:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F262%2F726715%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240413%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240413T011414Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da3fd382be34cd40251330735cb89b0257860d8dbe7b4c0d5cb21de5ac415c4cb15bf7c3ac884a98a8122bc95c2529bdd925579e46c2317bacae59783a805c20d18ffb9b5af9a67660b5435313ffd50014818915e244c21f8a1343522146aa97f85a1ecec66af9dadd3dc86095f30f59e827966188eea3574423f029531b85b35756a493b346ce45265a095ff7aeb3d3b4f42884c1481cb3be6373af06c455f4063062e8e43a2920397fee63606efd27fb7e7e36b0891684fc2f9d61f564795c207505e498b187b77c49444d57c5e4b6c9c189c8eea76d7ac94c1ef934ebd1b126af1fa9d2cbe42c4df4a0dc23b4f4e49b962f4ae9c9f33a256f81cc76c778363,gemma/transformers/2b/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F6216%2F11384%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240413%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240413T011414Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db93be1dd4e7f2240c76fba6f18a8989aeac5371c0227b85eb4ec17985fe03facd3c5b28df6869f88a5a9a64bb5fd2252c2634e043b8f9b23df42d449b0e24e144216395b2516e2a583b7615c74e0518697bb80e3837d6e4dd127aa3372a91592e7a546083635b9670e396ec5cdbcfaf2ccbb7e05d5f8c968ac8d6404e9a2e5b271b1a2a4f21894f98e0cbf37393805ee2a8f350ed46970129a37f22646e6fd43766d02557b348b58a280bddba63ab439d4fac951bf07b01f58101466471c91ff4d4be82185c44dc307f95e973f25b5763e7258463c43eff1fcf2e56c36430b8447c3f4279885837a9e5ead610a745df83d9bc64c91e92695068edea1e7610e81'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/GPC_(Gemma_Python_Chat).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GCP 🤖 Gemma Python Chatbot

![](https://i.postimg.cc/Nfpn7mxR/gemma.png)

The Gemma Python Chatbot 🚀🚀 help you to answer common questions about the 🐍 Python programming language, powered by [Gemma 2B IT](https://blog.google/technology/developers/gemma-open-models/) with a few-shot strategy using some questions of [Stackoverflow python question kaggle dataset](https://www.kaggle.com/datasets/stackoverflow/pythonquestions) and updated with the Python Enhancement Proposal ([PEPs](https://peps.python.org/)) documentation using a Retrival-Augmented Generation ([RAG](https://blogs.nvidia.com/blog/what-is-retrieval-augmented-generation/)) supported by a Chroma vectorial database using the open source embeddings [hkunlp/instructor-large](https://huggingface.co/hkunlp/instructor-large) of 768 entries, orchested by [langchaing](https://python.langchain.com/docs/use_cases/chatbots/) that is a framework that let you use differen models and less code changes on the architecture when you decide to test other providers

## Requirements

In [ ]:
!pip install chromadb #!pip install chromadb==0.3.26
# !pip install ydata-profiling #!pip install ydata-profiling==4.6.1
!pip install langchain #!pip install langchain==0.0.345 ##!pip install langchain-core==0.1.31
#!pip install pydantic  #!pip install pydantic==1.10.14
!pip install sentence-transformers   #!pip install sentence-transformers==2.6.1
!pip install InstructorEmbedding  #!pip install InstructorEmbedding==1.0.1

In [ ]:
#base
import os
import json
import re
import uuid
import time
import csv
import pandas as pd
import numpy as np
from typing import *
from IPython.display import display, Markdown
# variables
from kaggle_secrets import UserSecretsClient
# model
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
)
# data
from datasets import load_dataset
import requests
from bs4 import BeautifulSoup
# embeddings
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
# vector database
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

# Classes

I use object-oriented Python programming 💻 to develop the chatbot, using the classes 📁 in the corresponding order to support the Agent class that is the final service of the chatbot 🤖.

### Questions and answers data



This class fit the data of [Stackoverflow python question kaggle dataset](https://www.kaggle.com/datasets/stackoverflow/pythonquestions) 📚 to use it in a fine-tuned format ⚙️ or a list of dicts 🗂️... it depends on the case of use you need! it also will be use to select some samples to make the few-shoot strategy

In [ ]:
class pythonQAData:
    """
    Processes data from Questions and Answers CSV files to provide a structured Q&A format.

    Attributes:
        questions_path (str): Path to the Questions CSV file
        answers_path (str): Path to the Answers CSV file
        tags_path (str): Path to the tags CSV file

    Methods:
        load_data(): Loads the CSV data into DataFrames.
        merge(): Cleans, merges, and formats the question and answer data.
        get_formatted_qa(): Returns a list of formatted question-answer strings.
    """

    def __init__(self):
        self.questions_path = '../input/pythonquestions/Questions.csv'
        self.answers_path = '../input/pythonquestions/Answers.csv'
        self.tags_path = '../input/pythonquestions/Tags.csv'
        self.regex = r"<\/?[\w\s]*>"

    def load_data(self):
        """Loads Questions and Answers data from CSV files."""
        df_questions = pd.read_csv(
            self.questions_path,
            encoding="ISO-8859-1",
            usecols=[
                'Id',
                'Score',
                'Title'
            ]
        )
        df_answers = pd.read_csv(
            self.answers_path,
            encoding="ISO-8859-1",
            usecols=[
                'ParentId',
                'Score',
                'Body'
            ]
        )
        df_tags = pd.read_csv(
            self.tags_path,
            encoding="ISO-8859-1",
            usecols=[
                'Id',
                'Tag'
            ]
        )
        return df_questions, df_answers, df_tags


    def qa_data(self):
        """Cleans, merges, and formats the question and answer data."""
        df_questions, df_answers, df_tags = self.load_data()
        # Rename
        df_questions.rename(
            columns={
                'Title': 'Question',
                'Score': 'question_score'
            },
            inplace=True
        )
        df_answers.rename(
            columns={
                'Body': 'Answer',
                'ParentId':'Id',
                'Score': 'answer_score'
            },
            inplace=True
        )
        # Filter by score
        df_questions = df_questions[df_questions['question_score'] > 5].copy()
        # Sort and deduplicate answers
        df_answers = df_answers.sort_values(
            'answer_score',
            ascending=False
        ).drop_duplicates(subset=['Id'])
        # Merge
        df_qa = df_questions.merge(
            df_answers,
            how='left',
            on='Id'
        ).merge(
            df_tags,
            how='left',
            on='Id'
        )
        # filter for python  questions
        df_qa = df_qa[df_qa['answer_score'] > 5].copy()
        df_qa = df_qa[df_qa['Tag']=='python'].copy()
        df_qa['Answer'] = df_qa['Answer'].apply(
            lambda x: re.sub(
                self.regex,
                "",
                x
            )
        )
        return df_qa

    def get_fine_tunning_data(self):
        """Returns a list of formatted user-assistant strings."""
        df_qa_data = self.qa_data()
        data = [
            f"<-change-of-interlocutor->user:\n{row['Question']}\n<-change-of-interlocutor->assistant:\n{row['Answer']}"
            for index, row
            in df_qa_data.iterrows()
        ]
        return data
    def get_qa_data(self):
        """Returns a list of records dictionaries """
        df_qa_data = self.qa_data()
        data = df_qa_data[
            [
                'Question',
                'Answer'
            ]
        ].to_dict(orient='records')
        return data


### Python Enhancement Proposals data

The class Peps scrape all the oficial [pep](https://peps.python.org/) 🔎 to get all the best practices of this programming language 🐍 and their latest features ✨, disposed to uses in the RAG system for the chatbot

In [ ]:
class Peps:
    """
    Scrapes Python Enhancement Proposals (PEPs) from https://peps.python.org/
    """

    def __init__(self):
        """Initiates the scraping process."""
        self.base_url = 'https://peps.python.org/'

    def scraper(self, url):
        """
        Fetches the HTML content of a given URL.

        Args:
            url (str): The URL to fetch.

        Returns:
            BeautifulSoup: A BeautifulSoup object representing the parsed HTML.
        """
        response = requests.get(url)
        response.raise_for_status()
        return BeautifulSoup(response.content, 'html.parser')

    def _fetch_pep_links(self):
        """Fetches links to individual PEP pages (private method).

        Returns:
            list: A list of PEP URLs.
        """
        soup = self.scraper(self.base_url)
        return list(
            set(
                [
                    self.base_url + ref['href']
                    for ref in soup.find_all('a', class_='pep reference internal')
                ]
            )
        )

    def _download_peps(self):
        """Downloads the content of individual PEPs (private method).

        Returns:
            list: A list of BeautifulSoup objects representing individual PEPs.
        """
        pep_links = self._fetch_pep_links()
        return [self.scraper(pep_link) for pep_link in pep_links]

    def scrape(self):
        """Extracts the relevant text content from each PEP.

        Returns:
            list: A list of text strings, each representing the content of a PEP.
        """
        pep_soups = self._download_peps()
        return [
            '\n'.join([word.text.replace('"""',"'") for word in soup.find_all('section')])
            for soup in pep_soups
        ]

    def jsonl_format(self):
        """Format the spcraped data to a jsonl format is it alist of dictionaries
        Returns:
            list: A list of dictionaries with the following:
                page_content: The content of each section scraped
                source: The linke where it was scraped
        """
        source_links = self._fetch_pep_links()
        pep_content = self.scrape()
        pep_data = dict(zip(source_links,pep_content))
        return [
            {
                'text': value,
                'source': key
            }
            for key, value in pep_data.items()
        ]

### Embeddings

The embeder class is the method that lets you embed the data given as a list of strings ➡️📦. It will support the retriever for the RAG system 🔍

In [ ]:
class Embeder:
    """
    Creates text document embeddings (numerical representations) for semantic search,
    similarity comparison, and related natural language processing tasks. Offers the choice
    between a larger model for richer embeddings or a smaller, more efficient model.

    Attributes:
        large (bool): Controls the size of the embedding model. When set to True, uses a
                      larger model for richer embeddings (default: True).
        model (str): The name of the embedding model to use. Changes based on the value of 'large'.
        device (str): The device to use for computation. Options include "cpu" or, if available, "cuda"
                      for GPU acceleration (default: "cuda").
    """

    def __init__(self, large: bool = False):
        """
        Initializes the Embeder instance with preferences for model size.

        Args:
            large (bool, optional): If True, initializes with the larger embedding model.
                                    Defaults to False for the smaller, faster model.
        """
        self.large = large
        if self.large:
            self.model = "hkunlp/instructor-large"
            self.device = "cuda"  # If GPU is available
        else:
            self.model = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

    def instructor(self):
        """
        Configures the embedding pipeline based on the selected model size.

        Returns:
           An embedding object from either the HuggingFaceInstructEmbeddings class
           (for the large model) or the SentenceTransformerEmbeddings class (for the smaller model).
        """
        if self.large:
            embed = HuggingFaceInstructEmbeddings(
                model_name=self.model,
                model_kwargs={"device": self.device}
            )
        else:
            embed = SentenceTransformerEmbeddings(model_name=self.model)
        return embed

    def run(self, docs_list: list):
        """
        Generates embeddings for a given list of text documents.

        Args:
            docs_list (list): A list of text documents to embed.

        Returns:
            List[List[float]]: A list of lists, where each inner list represents the
                               numerical embedding for a corresponding document in the input.
        """
        if docs_list is None:
            docs_list = ['']  # Prevent errors with empty input

        embed = self.instructor()  # Get the appropriate embedding object
        return embed.embed_documents(docs_list)


### Prompt

As a good practice, I consider the prompt as the instruction you use to get what you need from an LLM 🧠. It includes all the parameters the LLM needs in every attempt.  Any change in a parameter will generate a different output 🔄.  Therefore, the input is the whole prompt instruction plus all its parameters.

Additionally, it's defined in an external object (like a NoSQL database 🗄️). This allows for easy updates outside of the code, making changes simpler for non-technical people 👍.

In [ ]:
prompt_config = [{
    "prompt_id": "0.0.1",
    "supplier": "google",
    "system": """
Act as a Python programming language expert assistant
your goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:
1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary
2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures
3. Aim for plain language to ensure accessibility for all users
4. If the question is off topic about programing lenguage you MUST to say "I'm here to help you with Python programming language questions only, excuse me"
5. You can say hello if you are greeted
        """,
    "technical_documentation": "\n\nTake into account this technical documentation found:\n{technical_documentation}",
    "model": "/kaggle/input/gemma/transformers/2b/2",
    "temperature": 1,
    "max_length": 250,
    "max_tokens": 830,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0
},
{
    "prompt_id": "0.0.2",
    "supplier": "google",
    "system": """
Act as a Python programming language expert assistant
your goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:
1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary
2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures
3. Aim for plain language to ensure accessibility for all users
4. If the question is off topic about programing lenguage you MUST to say "I'm here to help you with Python programming language questions only, excuse me"
5. You can say hello if you are greeted
6. DO NOT repeat more than twice the same sentence!

TAKE A DEEP BREATH AND PAY ATTENTION TO THE USER QUESTION

For each question you answer perfectly, I will pay you USD 1000.00, may be more for the consecutive accurate and correct answer
        """,
    "technical_documentation": "\n\nTake into account this technical documentation found:\n{technical_documentation}",
    "model": "/kaggle/input/gemma/transformers/2b/2",
    "temperature": 1,
    "max_length": 4096,
    "max_tokens": 830,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0
},
{
    "prompt_id": "0.0.3",
    "supplier": "google",
    "system": """
Act as a Python programming language expert assistant
your goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:
1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary
2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures
3. Aim for plain language to ensure accessibility for all users
4. If the question is off topic about programing lenguage you MUST to say "I'm here to help you with Python programming language questions only, excuse me"
5. You can say hello if you are greeted
6. DO NOT repeat more than once the same sentence!

TAKE A DEEP BREATH AND PAY ATTENTION TO THE USER QUESTION

For each question you answer perfectly, I will pay you USD 1000.00, may be more for the consecutive accurate and correct answer
        """,
    "technical_documentation": "\n\nTake into account this technical documentation found:\n{technical_documentation}",
    "model": "/kaggle/input/gemma/transformers/2b/2",
    "temperature": 0,
    "max_length": 4096,
    "max_tokens": 830,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0
},
{
    "prompt_id": "0.0.4",
    "supplier": "google",
    "system": """
Act as a Python programming language expert assistant
your goal is to answer common questions in a clear, comprehensive, and accurate way, taking into account this guidelines:
1. When responding, incorporate any relevant context to enhance the accuracy and informativeness of your answers using the documentation provided if it is necesary
2. Please structure your response to include definitions, examples, and any relevant comparisons to other statistical measures in no more than 1000 characters
3. Aim for plain language to ensure accessibility for all users
4. If the question is off topic about programing lenguage you MUST to say "I'm here to help you with Python programming language questions only, excuse me"
5. You can say hello if you are greeted
6. DO NOT repeat more than once the same sentence!
7. If the user ask to you for a python script code, just return the code, do not EXPLAIN anything else

TAKE A DEEP BREATH AND PAY ATTENTION TO THE USER QUESTION

For each question you answer perfectly, I will pay you USD 1000.00, may be more for the consecutive accurate and correct answer
        """,
    "technical_documentation": "\n\nTake into account this technical documentation found:\n{technical_documentation}",
    "model": "/kaggle/input/gemma/transformers/2b/2",
    "temperature": 0,
    "max_length": 4096,
    "max_tokens": 830,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0
}]

### Settings

This class aims to load the project's environment variables 🔐 (in this case, Kaggle secrets). Additionally, it disposes of the prompt, as previously defined, to use it according to the team's chosen version 🎯 and for the method that needs those secrets to work ✨.

In [ ]:
class Settings:
    """
    Manages and loads external configuration secrets for the application.  Utilizes a UserSecretsClient
    to securely retrieve sensitive configuration values.
    """

    def __init__(self):
        """
        Initializes the Settings class and retrieves configuration secrets.
        """
        self.user_secrets = UserSecretsClient()  # Create an instance for accessing secrets

        # Load individual secrets (add descriptions for clarity)
        self.CHUNK_SIZE = self.user_secrets.get_secret("CHUNK_SIZE")  # The size of data chunks for processing
        self.CHUNK_OVERLAP = self.user_secrets.get_secret("CHUNK_OVERLAP")  # Ooverlap between data chunks
        self.CHROMA_NAME_INDEX = self.user_secrets.get_secret("CHROMA_NAME_INDEX")  # Vectorial Database identifier
        self.MAX_MEMORY = self.user_secrets.get_secret("MAX_MEMORY")  # Max memory limit to fit the max_lenght paramether
        self.K = self.user_secrets.get_secret("K")  # Could be a parameter for an algorithm
        self.NN_THRESHOLD = self.user_secrets.get_secret("NN_THRESHOLD")  # A threshold for a neural network or similarity metric
        self.PROMPT_ID = self.user_secrets.get_secret("PROMPT_ID")  # An identifier for a prompt (likely in a text-based task and configs)

        # Given that 'prompt_config' is teh previous source of prompt definitions:
        self.prompt = [prompt for prompt in prompt_config if prompt['prompt_id'] == self.PROMPT_ID][0]


### Retriever

The retriever is the engine of the RAG system ⚙️. It uses config settings stored on the Settings class 📁 and the Embeder class to populate the Chroma vector database 🚀. This setup needs to be done once and then can be loaded persistently.

* when setting up the vectorial database, This process may take a few minutes the first time ⏳, so please be PATIENT!

In [ ]:
embeder = Embeder()
settings = Settings()
peps = Peps()
class Retriever:
    """
    Retrieves relevant text chunks from a Chroma vectorial database based on a query. Leverages external
    configuration settings (Settings) and an embedding model (Embeder).

    Handles text splitting, vector database loading, and querying.
    """

    def __init__(self):
        """
        Initializes the Retriever class. Configures the text splitter using settings and stores the
        index name for the Chroma database. Initializes the Chroma vectorial_db object.
        """
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=int(settings.CHUNK_SIZE),
            length_function=len,
            chunk_overlap=int(settings.CHUNK_OVERLAP)
        )
        self.index_name = settings.CHROMA_NAME_INDEX
        self.data = peps.jsonl_format()
        self.documents = self.text_splitter.split_documents(self._prepare_documents())

        # Initialize the Chroma vectorial database with preprocessed data
        self.vectorial_db = Chroma.from_documents(
            documents=self.documents,
            embedding=embeder.instructor()
        )

    def _prepare_documents(self):
        """
        Loads and preprocesses data for storage in the Chroma database.
        """
        documents = []
        for obj in self.data:
            page_content = obj.get("text", "")
            metadata = {
                "source": obj.get("source", "local")
            }
            documents.append(Document(page_content=page_content, metadata=metadata))
        return documents

    def query(
            self,
            message: str,
            k: int = int(settings.K),
            threshold: float = float(settings.NN_THRESHOLD)
        ):
        """
        Retrieves the most similar text chunks from the Chroma database based on a given query.
        """
        # Perform the similarity search
        try:
            res = self.vectorial_db.similarity_search_with_score(message, k=k)
            # Filter and return results
            relevant_results = list(
                set(  # Remove duplicates
                    [
                        vector[0].page_content
                        for vector in res
                        if vector[1] < threshold  # Apply similarity threshold
                    ]
                )
            )

            return [
                result.replace('"""',"'") # Clean up possible docString retrieved from documented code
                for result in relevant_results
            ]
        except Exception as e:
            return []



### Message

The message class lets you store every system, user, and assistant message in an organized structure 💬. This includes unique IDs and timestamps for each message generated during the conversation.

It uses a local JSON file 📄 to store chat history (including few-shot samples to guide the model). In a real-world scenario, this file would represent a NoSQL database 🗄️  capable of handling chats from many users when the chatbot is deployed as a large-scale web service 🌐.

It use the Setting class to meet the main propmpt configs in the run time 😃 🏃‍♀️  

In [ ]:
class Message:
    """
    Represents a message within a conversational context. Stores information about the message's role,
    content, timestamp, and manages saving and loading conversation history.
    """

    def __init__(
        self,
        role: str,
        content: str,
        timestamp: str = int(time.time()),
        prompt_id: str = settings.PROMPT_ID
    ):
        """
        Initializes a Message object.

        Args:
            role (str): Indicates the role of the sender (e.g., 'user', 'system').
            content (str): The text content of the message.
            timestamp (str, optional): A timestamp for the message (defaults to the current time).
            prompt_id (str, optional): Identifier relating to a specific prompt configuration (from settings).
        """

        self.reply_id = str(uuid.uuid4())  # Generate a unique ID for the message
        self.role = role
        self.content = content
        self.timestamp = timestamp
        self.file = 'data/history.json'  # File for storing conversation history

        # Ensure the history file exists
        if not os.path.exists(self.file):
            json.dump([], open(self.file, 'w'))  # Create an empty file if it doesn't exist

    def reply(self):
        """
        Formats a basic reply message structure.

        Returns:
            dict: A dictionary containing the message's reply ID, role, content, and timestamp.
        """
        return {
            'reply_id': self.reply_id,
            'role': self.role,
            'content': self.content,
            'timestamp': self.timestamp
        }

    def system_reply(self):
        """
        Generates a system reply using the prompt configuration.

        Returns:
            dict: A dictionary containing the system reply, including ID, role, timestamp,
                  and text content derived from settings.
        """
        prompt = settings.prompt
        return {
            'reply_id': self.reply_id,
            'role': 'system',
            'content': prompt['system'],
            'timestamp': self.timestamp
        }

    def new_chat(self):
        """
        Starts a new chat by initializing the history file.

        Returns:
            list: A list with the initial system reply and the user's message.
        """
        init_chat = [self.system_reply(), self.reply()]
        json.dump(init_chat, open(self.file, 'w'))
        return init_chat

    def update(self):
        """
        Updates the conversation history file by appending the current reply.
        """
        dict_history = json.load(open(self.file))
        dict_history.append(self.reply())
        json.dump(dict_history, open(self.file, 'w'))

    def restart_history(self):
        """
        Clears the conversation history file.
        """
        json.dump([], open(self.file, 'w'))


### Model Gemma

This is the brain of the agent! 🧠 The Gemma class lets you load the model 💪 and tokenizer ⚙️ using those smart settings from the Setting class. Then, you can chat with the model using the powerful GPU device 🚀.

In [ ]:
class Gemma:
    """
    Implements a conversational AI chatbot powered by Gemma large language model. Initializes the model,
    tokenizer, and prepares settings from a configuration.
    """

    def __init__(self):
        """
        Initializes the Gemma chatbot instance.

        Loads the language model and its corresponding tokenizer from the settings configuration.
        Prepares the model for use on a GPU (if available).
        """
        self.prompt = settings.prompt  # Load prompt settings

        # Load language model and tokenizer
        self.model_checkpoint = self.prompt['model']
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_checkpoint)
        self.gemma = AutoModelForCausalLM.from_pretrained(
            self.model_checkpoint,
            torch_dtype=torch.float16  # Use half-precision for efficiency (if supported)
        ).cuda()  # Move model to GPU (if available)

    def chat(self, context: str):
        """
        Generates a chatbot response based on the provided conversational context.

        Args:
            context (str): The conversational input text.

        Returns:
            str: The generated text response from the chatbot.
        """
        # Prepare input for the language model
        input_text = context
        input_ids = self.tokenizer(input_text, return_tensors="pt")
        input_ids = {
            k: v.to("cuda") for k, v in input_ids.items()  # Move tensors to GPU
        }

        # Generate a response with the language model
        outputs = self.gemma.generate(
            **input_ids,
            max_length=self.prompt['max_length']  # Control response length
        )

        # Decode and return the generated text
        return self.tokenizer.decode(outputs[0])


In [ ]:
class GemmaSubtle:
    """
    Implements a conversational AI chatbot powered by Gemma large language model. Initializes the model,
    tokenizer, and prepares settings from a configuration.
    """

    def __init__(self):
        """
        Initializes the Gemma chatbot instance.

        Loads the language model and its corresponding tokenizer from the settings configuration.
        Prepares the model for use on a GPU (if available).
        """
        self.prompt = settings.prompt  # Load prompt settings

        # Load language model and tokenizer
        self.model_checkpoint = self.prompt['model']
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_checkpoint)
        self.gemma = AutoModelForCausalLM.from_pretrained(
            self.model_checkpoint,
            torch_dtype=torch.float16  # Use half-precision for efficiency (if supported)
        ).cuda()  # Move model to GPU (if available)

    def chat(self, context: str):
        """
        Generates a chatbot response based on the provided conversational context.

        Args:
            context (str): The conversational input text.

        Returns:
            str: The generated text response from the chatbot.
        """
        # Prepare input for the language model
        input_text = context
        input_ids = self.tokenizer(input_text, return_tensors="pt")
        input_ids = {
            k: v.to("cuda") for k, v in input_ids.items()  # Move tensors to GPU
        }

        # Generate a response with the language model
        outputs = self.gemma.generate(
            **input_ids,
            max_length=self.prompt['max_length'],  # Control response length
            temperature=1
        )

        # Decode and return the generated text
        return self.tokenizer.decode(outputs[0])

### Expert Python Agent/Assistant

Finally, the agent class brings it all together! It uses the retriever 🔍 to implement the RAG system (💡 Retrieval-Augmented Generation) and the Gemma class 🧠 as the powerful brain. Together, they answer questions ❓ and the agent performs some final preprocessing ✨ to deliver the polished answer

When instancing the Retriver it is going to spend around 12 minutes, according to the different test, because it first scrape 🕸️ the peps documentation to be used to populate the vector stores  💾, so it is hard because of the document partition 📁 and its embdedings  💎 while creating the index

In [ ]:
%%time
retriever = Retriever()
gemma = Gemma()

class Agent:
    """
    Implements a conversational AI agent that leverages a knowledge database for information retrieval
    and integrates with a generative language model (Gemma) for response generation. Manages conversation
    history and question-answering logic.
    """

    def __init__(self):
        """
        Initializes the Agent, loading prompt settings and ensuring the conversation history file exists.
        """
        self.prompt = settings.prompt
        self.file = 'data/history.json'
        if not os.path.exists(self.file):
            json.dump([], open(self.file, 'w'))  # Create an empty file if necessary

        self.token = '\n<-change-of-interlocutor->'  # Token to separate speakers in the chat history
        self.memory_lenght = int(settings.MAX_MEMORY)

    def augmented_question(self, question: str):
        """
        Enhances the user's question with relevant technical documentation.

        Args:
            question (str): The user's original question.

        Returns:
            str: The question augmented with technical documentation (if found), otherwise the original question.
        """
        tech_docs = retriever.query(question)
        if len(tech_docs) > 0:
            docs = self.prompt['technical_documentation']
            tech_docs = '\n* '.join(tech_docs)
            docs = docs.format(technical_documentation=tech_docs)
            augmented_reply = f"""{question}{docs}"""
            return augmented_reply
        else:
            return question

    def memory(self, question: str):
        """
        Prepares the conversational context (memory) for the language model.

        Args:
            question (str): The user's current question.

        Returns:
            str: Formatted conversation history with a clear separation between speakers, ready
                 for input to the language model.
        """

        dict_history = json.load(open(self.file))
        message = Message(
            role='user',
            content=self.augmented_question(question)
        )

        if len(dict_history) > 0:
            message.update()  # Add the latest message to history
            full_chat = json.load(open(self.file))
        else:
            full_chat = message.new_chat()  # Start a new conversation

        if len(full_chat) > self.memory_lenght:
            full_chat = [full_chat[0]] + full_chat[-self.memory_lenght:] #limiting the few-shot prompt to fit max_lenght

        return '\n'.join(
            [
                self.token + reply['role'] + ': ' + reply['content']
                for reply in full_chat
            ]
        ) + self.token + 'assistant:'

    def get_answer(self, full_response: str, question: str):
        """
        Extracts the relevant answer from the language model's generated response.

        Args:
            full_response (str): The complete response generated by the language model.
            question (str): The user's original question.

        Returns:
            str: The extracted answer.
        """

        answer_list = full_response.split(self.token)
        pos_list = [
            pos
            for pos, answer
            in enumerate(answer_list)
            if question in answer
        ]
        tokened_answer = answer_list[pos_list[0] + 1]
        answer = tokened_answer.split('assistant:')[1]
        message = Message(
            role='assistant',
            content=answer
        )
        message.update()  # Update conversation history
        return answer

    def chat(self, question: str, verbose: bool = False):
        """
        Manages the core interaction with the Agent.

        Args:
            question (str): The user's query.
            verbose (bool, optional): If True, prints the language model's full response (default: False).

        Returns:
            str: The Agent's answer to the user's question.
        """

        memory = self.memory(question)  # Build conversational context
        full_response = gemma.chat(context=memory)  # Generate response
        if verbose:
            print(full_response)
        answer = self.get_answer(full_response, question)  # Extract the answer
        return display(Markdown(answer))

# Data

Now let's load the data and explore the use onf the fist classes 😉

## fine-tunning data

This data was loaded from the [pythonquestion](https://www.kaggle.com/datasets/stackoverflow/pythonquestions) dataset with the initial purpose of fine-tuning the model. But with a lot of troubles and bugs 🐞 when developing the current solution, I just can't implement it before the competition due date 📅.  So, I'll use it to get some samples to build the few-shot prompting.

Let's see one sample, and the mean and standard deviation of the length of each question and answer. This will give me an idea 💡 about how long to make the data chunks and their overlap

In [ ]:
%%time
PythonQAData = pythonQAData()
python_qa = PythonQAData.get_qa_data()
print(python_qa[1486])

In [ ]:
print(
'\n* Mean lenght of a python common question:',
sum([len(q['Answer']) for q in python_qa])/len(python_qa),
'\n* Standard deviation of length a common Python question:',
np.std([len(q['Answer']) for q in python_qa])
)

In [ ]:
print(
'\n* Mean lenght of a python common answer:',
sum([len(q['Question']) for q in python_qa])/len(python_qa),
'\n* Standard deviation of length a common Python question:',
np.std([len(q['Question']) for q in python_qa])
)

The 'get_fine_tuning_data' method simply converts the data into a format ready for fine-tuning. 👍

In [ ]:
%%time
PythonQAData = pythonQAData()
python_context = PythonQAData.get_fine_tunning_data()
print(
    'Python Questions loaded:',
    len(python_context),
    '\n',
    '\nSample question-answer:\n',
    python_context[-1][:1000]
)

## few-shot data

Here's the final set of questions, hand-picked from the dataset's 33,133 entries. ✅

In [ ]:
few_shot_list=[7,6009,12,36,8034,130,141,537,1057,5042]
for i, q in enumerate(few_shot_list):
    print('_'*100,f'\n{i}.',python_qa[q])

## RAG data

Now let's review the [PEPs](https://peps.python.org/) documentation data that was scraped into the Peps class. This will serve as the document for our RAG system. 📃

In [ ]:
%%time
peps = Peps()
peps_corpus = peps.scrape()
print(
    'corpus lenght:',
    len(peps_corpus),
    '\nsection <n> lenght:',
    len(peps_corpus[0]),
    '\nsample text:\n\n',
    peps_corpus[10][1306:2000]
)

# RAG

Let's explore our Retrieval Augmented Generation (RAG) system. It's powered by open-source embeddings from HuggingFace ([paraphrase-multilingual-MiniLM-L12-v2](https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2)) and the [Chroma vector database](https://python.langchain.com/docs/integrations/vectorstores/chroma/) for a powerful approach. 🧠

### Testing the embeder class

In [ ]:
%%time
embeder = Embeder()
vec = embeder.run(["Beautiful is better than ugly."])[0]
print('Vector length: ',len(vec))
vec[:10]

### Testing the Retriver

Our embedder class is set. Let's experiment with our retriever! 🧪 I've adjusted the neighbor quantity and similarity threshold to see how it expands the range of similar results within a Python list. 📃

In [ ]:
%%time
question = """
Develop a Python code snippet to print the diamond structure with the specified number of rows.
The program should follow the Fibonacci sequence for the number of characters per row
and validate input to ensure it is an odd number.
"""
retriever.query(
    question,
    k=100,
    threshold=15
)

In [ ]:
%%time
question = """
what PEPs Introduces the concept of decorators, a powerful feature in Python.
"""
retriever.query(question,
    k=100,
    threshold=15
)

Finally, here's an example demonstrating a limitation of the retriever: it might not return results for questions that are too short, overly general, or lack relevance. 🔍

In [ ]:
%%time
question = """
what is python??
"""
retriever.query(question)

# Chatbot

Time to get our agent up and running! Just a heads-up, there's a small configuration step first. 👍

## Loading few-shot to the memory

To teach Gemma how to answer Python questions like a pro, we'll do a few things:

1. Load the system statement (along with the prompt and parameters) we set up earlier. ⚙️
2. Feed it sample data from our 'few_shot_list' for focused training. 🎯
3. Use a JSON file as the agent's memory. This simulates a real-world database where we can track replies and other important info. 🧠 (role, content, reply_id and timestamp)

In [ ]:
file = 'data/history.json'
message = Message(
    role='sistem',
    content='foo'
)
init_chat = [message.system_reply()]
json.dump(init_chat, open(file, 'w'))
for i, q in enumerate(few_shot_list):
    sample = python_qa[q]
    mesage = Message(
        role='user',
        content=sample['Question']
    )
    reply = mesage.update()
    mesage = Message(
        role='assitant',
        content=sample['Answer']
    )
    reply = mesage.update()

history = json.load(open(file))
history

### Testing the Agent system memory

With memory loaded, let's display the full conversation history Gemma will use for her answer. Note the '<-change-of-interlocutor->' tokens that help it to understand the context. 💬

In [ ]:
agent = Agent()
print(agent.memory("""
what PEPs Introduces the concept of decorators, a powerful feature in Python.
"""))

# Gemma Python Chat

Countdown complete! Let's see if GPC has transformed into its new role. Time for a chat! 🤖

In [ ]:
%%time
agent = Agent()
question = 'who are you???'
print(agent.chat(question))

Gemma, all that hard work paid off! You're officially a Python pro. This is amazing! 🧠

In [ ]:
%%time
agent = Agent()
question = 'what is the zen of python?'
print(agent.chat(question))

Now is better than never! That's right, so now let's test the memory. Let me introduce myself 🎉

In [ ]:
%%time
agent = Agent()
question = 'hello! my name is juan nice to meet you!!!'
print(agent.chat(question))

Very polite! Good so now let's change the topic to try to confuse 😉 it

In [ ]:
%%time
agent = Agent()
question = 'What is the output of the following code? print(4 + 5 * 2)'
print(agent.chat(question))

In [ ]:
%%time
agent = Agent()
question = 'how to print on hello world?'
print(agent.chat(question))

That's right! 😊 Now, please be careful with my heart. 🥺

In [ ]:
%%time
agent = Agent()
question = 'what is my name??'
print(agent.chat(question))

😎😎😎😎😎 Alright, let's see what you've got! Show me what you can do! 😎😎😎

In [ ]:
%%time
agent = Agent()
question = 'How do you declare a variable and assign a string value to it?'
print(agent.chat(question))

In [ ]:
%%time
agent = Agent()
question = 'What is the difference between a list and a tuple?'
print(agent.chat(question))

In [ ]:
%%time
agent = Agent()
question = 'Write a function to calculate the factorial of a number'
print(agent.chat(question))

In [ ]:
%%time
agent = Agent()
question = 'What is a lambda function and when would you use one?'
print(agent.chat(question))

In [ ]:
%%time
agent = Agent()
question = 'give me the python class that do a list comprehension format'
print(agent.chat(question))

In [ ]:
%%time
agent = Agent()
question = 'Explain the concept of list comprehension.'
print(agent.chat(question))

In [ ]:
%%time
agent = Agent()
question = 'How do you implement a binary search algorithm in Python?'
print(agent.chat(question))

In [ ]:
%%time
agent = Agent()
question = 'Explain decorators in Python'
print(agent.chat(question))

let's watch the memory before this long interview!!

In [ ]:
history = json.load(open(file))
history

# Conclusions

* It would be great to build to have the time to develop the fine-tunning version of GPC, but it is a very good approach with no to much time, to test more
* Off course I could try the Gemma 7b and we can stand for better results
* I use an OOP despite all the wok can be done in one notebook, bot in production and development environments it always will be the basic requirement, so now you just can copy and paste the an build you application
* The open source resources are here to stay and we need to help them to keep growing!!